In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/patch_gastric.zip" -d "./patch_gastric"

Archive:  /content/drive/MyDrive/patch_gastric.zip
 extracting: ./patch_gastric/captions.csv  
 extracting: ./patch_gastric/patches_captions.zip  


In [ ]:
!unzip -q "/content/patch_gastric/patches_captions.zip" -d "/content/patch_gastric/"
!rm "/content/patch_gastric/patches_captions.zip"

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import models
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from sentence_transformers import SentenceTransformer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class PatchGastricMILDataset(Dataset):
    def __init__(self, image_dir, label_csv, transform=None):
        self.image_dir = image_dir
        self.labels_df = pd.read_csv(label_csv)

        # Column names in your CSV
        scan_col = "id"      # <<< CHANGE THIS if needed
        label_col = "subtype"       # <<< CHANGE THIS if needed

        self.transform = transform if transform else transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ])

        # Group patch paths by scan_id
        self.scan_to_images = defaultdict(list)
        for fname in os.listdir(image_dir):
            if fname.endswith(".jpg"):
                scan_id = fname.split("_")[0]  # Extract scan_id from file name
                self.scan_to_images[scan_id].append(os.path.join(image_dir, fname))

        # Keep only those scan_ids that appear in both CSV and image set
        self.scan_ids = [sid for sid in self.labels_df[scan_col] if sid in self.scan_to_images]

        # Convert labels to numerical format
        unique_labels = self.labels_df[label_col].unique()
        self.label_to_int = {label: i for i, label in enumerate(unique_labels)}
        self.label_map = dict(zip(self.labels_df[scan_col], self.labels_df[label_col].map(self.label_to_int)))


    def __len__(self):
        return len(self.scan_ids)

    def __getitem__(self, idx):
        scan_id = self.scan_ids[idx]
        img_paths = self.scan_to_images[scan_id]

        images = []
        for path in img_paths:
            img = Image.open(path).convert("RGB")
            img = self.transform(img)
            images.append(img)

        bag_tensor = torch.stack(images)  # Shape: (num_patches, 3, H, W)
        label = torch.tensor(self.label_map[scan_id], dtype=torch.long)

        return bag_tensor, label, scan_id

In [ ]:
image_dir = "/content/patch_gastric/patches_captions"
label_csv = "/content/captions_filtered.csv"
feature_dir = "features_of_scans"

os.makedirs(feature_dir, exist_ok = True)

In [ ]:
dataset = PatchGastricMILDataset(image_dir, label_csv)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

In [ ]:
"FEATURE EXTRACTION OF HISTOPATHOLOGICAL IMAGES"

model = models.resnet50(pretrained=True)
model = nn.Sequential(*list(model.children())[:-1])
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

patch_batch_size = 64

for bag_tensor, label, scan_id in tqdm(dataloader):
    scan_id = scan_id[0]
    bag_tensor = bag_tensor.squeeze(0).to(device)  # shape: (#patches, 3, 224, 224)

    all_features = []
    for i in range(0, bag_tensor.size(0), patch_batch_size):
        patch_batch = bag_tensor[i : i + patch_batch_size]
        with torch.no_grad():
            features = model(patch_batch)  # shape: (batch_size, 2048, 1, 1)
            features = features.squeeze(-1).squeeze(-1)      # shape: (batch_size, 2048)
        all_features.append(features.cpu())

    scan_features = torch.cat(all_features, dim=0)

    # Save each scan's features to a file
    torch.save({
        "scan_id": scan_id,
        "label": label.item(),
        "features": scan_features
    }, os.path.join(feature_dir, f"{scan_id}.pt"))

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 836/836 [28:17<00:00,  2.03s/it]


In [ ]:
!zip -r features_per_scan_id.zip features_of_scans/

	zip warning: name not matched: features_of_scans/

zip error: Nothing to do! (try: zip -r features_per_scan_id.zip . -i features_of_scans/)


In [ ]:
!cp features_per_scan_id.zip /content/drive/MyDrive/

In [ ]:
captions = pd.read_csv("/content/captions_filtered.csv")
scan_col = "id"
caption_col = "text"

scan_to_caption_mapping = dict(zip(captions[scan_col], captions[caption_col]))

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
model.eval()

os.makedirs("features_per_caption", exist_ok=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
for _, _, scan_id in tqdm(dataloader):
    scan_id = scan_id[0]

    if scan_id not in scan_to_caption_mapping:
        continue

    caption = scan_to_caption_mapping[scan_id]

    # Skip if already processed
    out_path = f"features_per_caption/{scan_id}.pt"
    if os.path.exists(out_path):
        continue

    with torch.no_grad():
        embedding = model.encode(caption, convert_to_tensor=True)

    torch.save({
        "scan_id": scan_id,
        "embedding": embedding.cpu()
    }, out_path)

100%|██████████| 836/836 [17:47<00:00,  1.28s/it]


In [ ]:
!zip -r features_per_caption.zip features_per_caption//

  adding: features_per_caption// (stored 0%)
  adding: features_per_caption//ab8608572a514803b9707decf936c6f6.pt (deflated 31%)
  adding: features_per_caption//17625cc91f11415e91b0d0357967bdbd.pt (deflated 31%)
  adding: features_per_caption//9adf8947314b4fba85c754eb4c07bb5a.pt (deflated 31%)
  adding: features_per_caption//971e40c711434a8e9469af5d7938f3f2.pt (deflated 31%)
  adding: features_per_caption//c100a9f741f0475a875e20f2467e1e86.pt (deflated 31%)
  adding: features_per_caption//47dad51a913d4a6487b03eca9c41770a.pt (deflated 31%)
  adding: features_per_caption//0dee732d4534462d90dd57bc0cc1d1f1.pt (deflated 31%)
  adding: features_per_caption//3cc6b9c635164213bd03c97b142768b7.pt (deflated 31%)
  adding: features_per_caption//b6477df33dc2406886f821f616e1c854.pt (deflated 31%)
  adding: features_per_caption//2a0ced28198047fbac54cf8645b0c074.pt (deflated 31%)
  adding: features_per_caption//0370d465854043cba4062f6c694c45b4.pt (deflated 31%)
  adding: features_per_caption//3455f0c577

In [ ]:
!cp features_per_caption.zip /content/drive/MyDrive/

In [ ]:
class LMF(nn.Module):
    def __init__(self, rank=4, hidden_dims=[2048, 384], output_dim=128):
        super(LMF, self).__init__()
        self.rank = rank
        self.dim1 = hidden_dims[0] + 1  # +1 for bias
        self.dim2 = hidden_dims[1] + 1
        self.output_dim = output_dim

        self.factor1 = nn.Parameter(torch.randn(rank, self.dim1, output_dim))
        self.factor2 = nn.Parameter(torch.randn(rank, self.dim2, output_dim))

    def forward(self, x1, x2):
        # x1: [B, dim1], e.g. image features
        # x2: [B, dim2], e.g. caption features
        B = x1.size(0)

        ones = x1.new_ones(B, 1)
        x1 = torch.cat([x1, ones], dim=1)  # [B, dim1+1]
        x2 = torch.cat([x2, ones], dim=1)  # [B, dim2+1]

        fusion = 0
        for i in range(self.rank):
            proj1 = torch.matmul(x1, self.factor1[i])  # [B, output_dim]
            proj2 = torch.matmul(x2, self.factor2[i])  # [B, output_dim]
            fusion += proj1 * proj2  # Hadamard product

        return fusion  # [B, output_dim]

In [ ]:
class MultimodalClassifier(nn.Module):
    def __init__(self, rank=4, fusion_dim=128, num_classes=2):
        super().__init__()
        self.lmf = LMF(rank=rank, hidden_dims=[2048, 384], output_dim=fusion_dim)
        self.classifier = nn.Sequential(
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(fusion_dim, num_classes)
        )

    def forward(self, img_feat, cap_feat):
        fused = self.lmf(img_feat, cap_feat)
        return self.classifier(fused)


In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, dim, max_l = 1000):
        super().__init__()
        self.pe = nn.Parameter(torch.randn(max_l, dim))

    def forward(self, x):
        return x + self.pe[:x.size(1)]

In [ ]:
class AttentionPooling(nn.Module):
    def __init__(self, dim=2048, head=4):
        super().__init__()
        self.pos_enc = PositionalEncoding(dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=dim, nhead=head, batch_first=True)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=1)

    def forward(self, x):
        x = self.pos_enc(x)
        x = self.encoder(x)

        return x.mean(dim=1)

In [ ]:
model = MultimodalClassifier(rank=64, fusion_dim=128, num_classes=3)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
attention_pool = AttentionPooling()
attention_pool.to(device)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MultimodalClassifier(
  (lmf): LMF()
  (classifier): Sequential(
    (0): ReLU()
    (1): Dropout(p=0.3, inplace=False)
    (2): Linear(in_features=128, out_features=3, bias=True)
  )
)

In [ ]:
df = pd.read_csv("captions_filtered.csv")

train_df, test_df = train_test_split(
    df, test_size=0.2, stratify=df["subtype"], random_state=42
)
train_df.to_csv("train.csv", index=False)
test_df.to_csv("test.csv", index=False)

In [ ]:
!unzip "/content/drive/MyDrive/features_per_scan_id.zip" -d "/content/"
!unzip "/content/drive/MyDrive/features_per_caption.zip" -d "/content/"

Archive:  /content/drive/MyDrive/features_per_scan_id.zip
   creating: /content/features_of_scans/
  inflating: /content/features_of_scans/4ed5ca3865b84a53b8e71ee366d50cf0.pt  
  inflating: /content/features_of_scans/94ad05dd99854167a7839665582eabb5.pt  
  inflating: /content/features_of_scans/d9367cbc190047f0958fdc29f65358c8.pt  
  inflating: /content/features_of_scans/5c221f066df34676b3c44529d7e58586.pt  
  inflating: /content/features_of_scans/99a998aad2534dfe929e4d18fc8f0c5e.pt  
  inflating: /content/features_of_scans/5dbc9864d0e84db4883797402d9e19d0.pt  
  inflating: /content/features_of_scans/1a74d24cc7f74a5eb6908194342074b2.pt  
  inflating: /content/features_of_scans/469d90c7a9684454b5fd0727848c3fdc.pt  
  inflating: /content/features_of_scans/b5d638d64ebe4bb4a88503ab30a75f46.pt  
  inflating: /content/features_of_scans/8bdce224bd3b4e649d53cd475176fcd5.pt  
  inflating: /content/features_of_scans/4967eac05e1b4bed8fcc272b91099636.pt  
  inflating: /content/features_of_scans/50a

In [ ]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

label_to_idx = {label: idx for idx, label in enumerate(sorted(train_df["subtype"].unique()))}
num_classes = len(label_to_idx)

In [ ]:
i_feat = torch.load("features_of_scans/9177a46b09984ef59ea9e088741e5074.pt")['features']
i_feat.shape

torch.Size([409, 2048])

In [ ]:
for epoch in range(10):
    model.train()
    total_loss = 0
    all_preds, all_labels = [], []

    for _, row in tqdm(train_df.iterrows(), total=len(train_df), desc=f"Epoch {epoch + 1}"):
        scan_id = row["id"]
        label = row["subtype"]

        try:
            img_patches = torch.load(f"features_of_scans/{scan_id}.pt")["features"]
            img_feat = attention_pool(img_patches.unsqueeze(0).squeeze(0))
            cap_feat = torch.load(f"features_per_caption/{scan_id}.pt")["embedding"]
        except Exception as e:
            print(f"Skipping {scan_id}: {e}")
            continue

        img_feat = img_feat.unsqueeze(0).to(device)
        cap_feat = cap_feat.unsqueeze(0).to(device)
        label = torch.tensor([label_to_idx[label]]).to(device)

        logits = model(img_feat, cap_feat)
        loss = criterion(logits, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pred = torch.argmax(logits, dim=1).item()
        all_preds.append(pred)
        all_labels.append(label.item())

    acc = accuracy_score(all_labels, all_preds)
    print(f"Epoch {epoch + 1} | Train Loss: {total_loss:.4f} | Train Accuracy: {acc:.2%}")

Epoch 1:   3%|▎         | 20/668 [00:00<00:07, 89.52it/s]

Skipping e3ace33d8de44cc1b254adcf8fede952: The size of tensor a (49) must match the size of tensor b (1000) at non-singleton dimension 0
Skipping 4b026e1b74d34f31af298f91923129a3: The size of tensor a (312) must match the size of tensor b (1000) at non-singleton dimension 0
Skipping bd81b503bdae4e97ab430ff89075424d: The size of tensor a (340) must match the size of tensor b (1000) at non-singleton dimension 0
Skipping ea789fce2fce4ca6936307aaba07f304: The size of tensor a (475) must match the size of tensor b (1000) at non-singleton dimension 0
Skipping a9213001d6b34ba2aa5e140d2586251d: The size of tensor a (517) must match the size of tensor b (1000) at non-singleton dimension 0
Skipping 5baced50e72b4002bd57810875c2be89: The size of tensor a (445) must match the size of tensor b (1000) at non-singleton dimension 0
Skipping 0cab1e8bd4e24de5a0d8883e29c7deec: The size of tensor a (80) must match the size of tensor b (1000) at non-singleton dimension 0
Skipping 1499abcfcbeb415f9cae49223cb

Epoch 1:  10%|▉         | 66/668 [00:00<00:03, 158.55it/s]

Skipping 0eae64f63f4a45c78983a47f674fb2b5: The size of tensor a (64) must match the size of tensor b (1000) at non-singleton dimension 0
Skipping 62fd8df4c0ff48d89c6f1e392e958f17: The size of tensor a (48) must match the size of tensor b (1000) at non-singleton dimension 0
Skipping ce51fbe04a3748b4998df3ff83beef3b: The size of tensor a (68) must match the size of tensor b (1000) at non-singleton dimension 0
Skipping 7f8e7f03ec63444abce61b8288134f12: The size of tensor a (123) must match the size of tensor b (1000) at non-singleton dimension 0
Skipping 6881f4d2e7144138acaac7fc03eeb5d8: The size of tensor a (109) must match the size of tensor b (1000) at non-singleton dimension 0
Skipping bcab6978ee9a4e72a6b202e613b1896a: The size of tensor a (87) must match the size of tensor b (1000) at non-singleton dimension 0
Skipping 7c9258a90a9c4b72835c30058a2662b3: The size of tensor a (113) must match the size of tensor b (1000) at non-singleton dimension 0
Skipping e761a64c5a5b4790bc75170acbd9a

Epoch 1:  17%|█▋        | 111/668 [00:00<00:03, 177.50it/s]

...

Epoch 10:  44%|████▍     | 295/668 [00:00<00:00, 586.48it/s]

Skipping c3d2d9e64e6b4db1ac1b3fd0e5090227: The size of tensor a (528) must match the size of tensor b (1000) at non-singleton dimension 0
Skipping 9608db80fbdb4c709829a3de4de72dd5: The size of tensor a (229) must match the size of tensor b (1000) at non-singleton dimension 0
Skipping b7f36de5028e4a68a87444cdf1dd4461: The size of tensor a (83) must match the size of tensor b (1000) at non-singleton dimension 0
Skipping a20d17b1fc9941ee8ddfb9bf5ba71bd9: The size of tensor a (187) must match the size of tensor b (1000) at non-singleton dimension 0
Skipping 58cc0e4a334c4f7085c2d92bba308322: The size of tensor a (62) must match the size of tensor b (1000) at non-singleton dimension 0
Skipping 55b60ebd522c48f5b9daf891c3aa7be7: The size of tensor a (495) must match the size of tensor b (1000) at non-singleton dimension 0
Skipping 45d3881171d44e9094ee19c697bd3792: The size of tensor a (144) must match the size of tensor b (1000) at non-singleton dimension 0
Skipping ad607d89def147d7a957e48b410

Epoch 10:  55%|█████▍    | 366/668 [00:00<00:00, 579.88it/s]

Skipping 2d2ca384a3de4d778e8ceddb6b3002b3: The size of tensor a (145) must match the size of tensor b (1000) at non-singleton dimension 0
Skipping 59893bbcc7fd49e7b3b58ee0fe5dc156: The size of tensor a (65) must match the size of tensor b (1000) at non-singleton dimension 0
Skipping 642147765a8c4b0dafae19c2a4a83898: The size of tensor a (364) must match the size of tensor b (1000) at non-singleton dimension 0
Skipping 0f440ab233c9479492c339b240977d10: The size of tensor a (195) must match the size of tensor b (1000) at non-singleton dimension 0
Skipping 83c45f45d1404cdbafea01937de15276: The size of tensor a (826) must match the size of tensor b (1000) at non-singleton dimension 0


KeyboardInterrupt: 

In [ ]:
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for _, row in test_df.iterrows():
        scan_id = row["id"]
        label = row["subtype"]

        try:
            img_feat = torch.load(f"features_of_scans/{scan_id}.pt")["features"].mean(dim=0)
            cap_feat = torch.load(f"features_per_caption/{scan_id}.pt")["embedding"]
        except:
            continue

        img_feat = img_feat.unsqueeze(0).to(device)
        cap_feat = cap_feat.unsqueeze(0).to(device)

        logits = model(img_feat, cap_feat)
        pred = torch.argmax(logits, dim=1).item()

        all_preds.append(pred)
        all_labels.append(label_to_idx[label])

test_acc = accuracy_score(all_labels, all_preds)
print(f"Final Test Accuracy: {test_acc:.2%}")

Final Test Accuracy: 86.31%


In [ ]:
print(all_preds)
print(all_labels)

[0, 0, 2, 0, 0, 0, 2, 2, 2, 0, 0, 2, 0, 0, 2, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 2, 1, 1, 2, 2, 2, 2, 2, 0, 2, 0, 2, 1, 2, 0, 1, 2, 2, 0, 0, 2, 2, 0, 0, 2, 0, 1, 2, 2, 0, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 0, 2, 2, 0, 2, 2, 2, 1, 0, 0, 0, 2, 0, 2, 2, 0, 1, 1, 0, 1, 0, 0, 1, 2, 2, 2, 1, 0, 2, 2, 0, 0, 2, 1, 0, 2, 0, 2, 1, 1, 0, 2, 1, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 2, 1, 2, 2, 0, 0, 2, 0, 0, 1, 1, 0, 0, 1, 2, 1, 2, 0, 0, 2, 1, 0, 0, 0, 2, 1, 0, 0, 2, 2, 1, 0, 1, 0, 0]
['Well differentiated tubular adenocarcinoma', 'Moderately differentiated tubular adenocarcinoma', 'Well differentiated tubular adenocarcinoma', 'Well differentiated tubular adenocarcinoma', 'Moderately differentiated tubular adenocarcinoma', 'Moderately differentiated tubular adenocarcinoma', 'Well differentiated tubular adenocarcinoma', 'Moderately differentiated tubular adenocarcinoma', 'Well differentiated tubular adenocarcinoma', 'Moderately differentiated tubular adenocarcinoma', 'Mod